   * https://blog.min.io/kafka_and_minio/: Publish from Kafka, Persist on MinIO
   * https://docs.confluent.io/kafka-connectors/s3-sink/current/overview.html#
      * Search "Quick Start"  

In [1]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"s3-sink-cdc","namespace":"kafka-main-cluster"},"spec":{"class":"io.confluent.connect.s3.S3SinkConnector","config":{"consumer.security.protocol":"SSL","consumer.ssl.enabled.protocols":"TLSv1.2,TLSv1.1","consumer.ssl.key.password":123456,"consumer.ssl.keystore.location":"/opt/kafka/external-configuration/self-signed-certificate/cluster.truststore.jks","consumer.ssl.keystore.password":123456,"consumer.ssl.truststore.type":"JKS","flush.size":1,"format.class":"io.confluent.connect.s3.format.avro.AvroFormat","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","key.converter.schemas.enable":true,"pa

In [19]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.conditions[0].message}'

The following tasks have failed: 0, see connectorStatus for more details.

In [3]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.connectorStatus.tasks[0].trace}'

# Self signed certificate

In [4]:
!echo | openssl s_client -connect minio.minio-tenant-1.svc:443

Connecting to 10.102.87.93
CONNECTED(00000003)
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=20:unable to get local issuer certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=21:unable to verify the first certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify return:1
---
Certificate chain
 0 s:O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
   i:CN=minikubeCA
   a:PKEY: id-ecPublicKey, 256 (bit); sigalg: RSA-SHA256
   v:NotBefore: Feb 20 11:55:36 2024 GMT; NotAfter: Feb 19 11:55:36 2025 GMT
---
Server certificate
-----BEGIN CERTIFICATE-----
MIIDnzCCAoegAwIBAgIQJl/BrEYomZbpipLiC0N6ujANBgkqhkiG9w0BAQsFADAV
MRMwEQYDVQQDEwptaW5pa3ViZUNBMB4XDTI0MDIyMDExNTUzNloXDTI1MDIxOTEx
NTUzNlowYjEVMBMGA1UEChMMc3lzdGVtOm5vZGVzMUkwRwYDVQQDDEBzeXN0ZW06
b

In [5]:
!echo | openssl s_client  -connect minio.minio-tenant-1.svc:443 2> /dev/null \
      | sed -ne '/-BEGIN CERTIFICATE-/,/-END CERTIFICATE-/p'

-----BEGIN CERTIFICATE-----
MIIDnzCCAoegAwIBAgIQJl/BrEYomZbpipLiC0N6ujANBgkqhkiG9w0BAQsFADAV
MRMwEQYDVQQDEwptaW5pa3ViZUNBMB4XDTI0MDIyMDExNTUzNloXDTI1MDIxOTEx
NTUzNlowYjEVMBMGA1UEChMMc3lzdGVtOm5vZGVzMUkwRwYDVQQDDEBzeXN0ZW06
bm9kZToqLm1pbmlvLXRlbmFudC0xLWhsLm1pbmlvLXRlbmFudC0xLnN2Yy5jbHVz
dGVyLmxvY2FsMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE2H7LRubBL5CvoonT
qjMie9ixVIY9SHPc3Ef6buJ71tLLKX6ypdF0OFoaT8zcz5K2YZiluEtDd0QQG/BK
vqBOLaOCAWcwggFjMA4GA1UdDwEB/wQEAwIFoDATBgNVHSUEDDAKBggrBgEFBQcD
ATAMBgNVHRMBAf8EAjAAMB8GA1UdIwQYMBaAFNzr9WFJe8Ffg8lMFwmUzPqVrWpZ
MIIBCwYDVR0RBIIBAjCB/4JObWluaW8tdGVuYW50LTEtc3MtMC17MC4uLjJ9Lm1p
bmlvLXRlbmFudC0xLWhsLm1pbmlvLXRlbmFudC0xLnN2Yy5jbHVzdGVyLmxvY2Fs
giVtaW5pby5taW5pby10ZW5hbnQtMS5zdmMuY2x1c3RlcjIueHB0ghRtaW5pby5t
aW5pby10ZW5hbnQtMYIYbWluaW8ubWluaW8tdGVuYW50LTEuc3ZjgjMqLm1pbmlv
LXRlbmFudC0xLWhsLm1pbmlvLXRlbmFudC0xLnN2Yy5jbHVzdGVyMi54cHSCISou
bWluaW8tdGVuYW50LTEuc3ZjLmNsdXN0ZXIyLnhwdDANBgkqhkiG9w0BAQsFAAOC
AQEAlO3AECYIjWLr4M4Q2TkrJEramrlpB96aevkDw00exCWHdD6G4GenIqCVD8